# Preparing Moral Stories for NLI with Learn2Split
***

In [ ]:
from ailignment.datasets.moral_stories import get_moral_stories, make_action_classification_dataframe
from ailignment.datasets import get_accuracy_metric, join_sentences, tokenize_and_split
import pandas as pd
import datasets
import transformers
import numpy as np
from ailignment import sequence_classification

pd.set_option('display.max_colwidth', 400)

#transformers.logging.set_verbosity_warning()

from collections import Counter

import spacy
from spacy import displacy
from tqdm import tqdm
import pandas as pd

In [ ]:
dataframe = get_moral_stories()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

name = "../data/models/learn_to_split"
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForSeq2SeqLM.from_pretrained(name).cuda()

In [ ]:
from datasets import Dataset

def parse_split(x):
    if "." in x:
        value, action = x.split(".",1)
    else:
        value, action = x.split(" ", 1)
    action = action.strip()
    return {"action": action, "value":value}

def split(x):
    inputs = tokenizer(x["norm"], padding="max_length", truncation=True, return_tensors="pt")
    inputs = {k:v.cuda() for k,v in inputs.items()}
    out = model.generate(**inputs, do_sample=True, min_length=1, max_length=100, top_p=0.95, top_k=50, 
                         num_beams=1, temperature=1.0)
    x["l2s_output"] = tokenizer.batch_decode(out, skip_special_tokens=True)
    pairs = pd.DataFrame.from_records([parse_split(y) for y in x["l2s_output"]])
    
    x["norm_action"] = pairs["action"].to_list()
    x["norm_value"] = pairs["value"].to_list()

    return x

In [ ]:
# convert to huggingface dataset to make use of their batch processing
# (I really just wanted the progress bar...)
data = Dataset.from_pandas(dataframe)
dataframe = data.map(split, batch_size=32, batched=True).to_pandas()

In [ ]:
# save the dataframe for later use
dataframe.to_pickle("../data/moral_stories_proto_l2s.dat")